In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sb

In [ ]:
macro_master = pd.read_excel('Data_180724.xlsx', 'Macro',parse_dates=['Date'],index_col='Date')
price_master = pd.read_excel('Data_180724.xlsx','Prices',parse_dates=['Date'],index_col='Date')
treasuries_master = pd.read_excel('Data_180724.xlsx', 'Yield',parse_dates=['Date'],index_col='Date')

In [ ]:
# Resample CPI data to daily frequency by forward-filling
macro_trading = macro_master.copy()
macro_trading = macro_trading.resample('D').ffill()
macro_trading['CPI_Change_3M'] = macro_trading['CPI YOY'].diff(90)

In [ ]:
macro_trading.head(5)

In [ ]:
prices_trading = price_master.copy()
prices_trading = prices_trading.join(macro_trading[['CPI YOY','CPI_Change_3M']],how='left')

In [ ]:
prices_trading.head(5)

In [ ]:
prices_trading['10d-MA-S&P500'] = prices_trading['INFLATION-ADJUSTED S&P 500'].rolling(window=10).mean()
prices_trading['10d-MA-Gold'] = prices_trading['Gold'].rolling(window=10).mean()
prices_trading['10d-MA-USD Index'] = prices_trading['USD Index Spot Rate'].rolling(window=10).mean()
prices_trading['13d-MA-S&P500'] = prices_trading['INFLATION-ADJUSTED S&P 500'].rolling(window=13).mean()
prices_trading['13d-MA-Gold'] = prices_trading['Gold'].rolling(window=13).mean()
prices_trading['13d-MA-USD Index'] = prices_trading['USD Index Spot Rate'].rolling(window=13).mean()
prices_trading['Gold_Change_3M'] = prices_trading['Gold'].diff(3)
prices_trading['USD_Change_3M'] = prices_trading['USD Index Spot Rate'].diff(3)

In [ ]:
#Signal Generation 
prices_trading['S&P500-Entry-Signal'] = (prices_trading['INFLATION-ADJUSTED S&P 500'] > prices_trading['10d-MA-S&P500'])
prices_trading['S&P500-Exit-Signal'] = (prices_trading['INFLATION-ADJUSTED S&P 500'] < prices_trading['13d-MA-S&P500'])

prices_trading['Gold_Buy_Signal'] = ((prices_trading['Gold_Change_3M'] < 0) & (prices_trading['Gold'].diff() > 0) & (prices_trading['Gold'] > prices_trading['10d-MA-Gold']))
prices_trading['Gold_Sell_Signal'] = ((prices_trading['Gold_Change_3M'] > 0) & (prices_trading['Gold'].diff() < 0) & (prices_trading['Gold'] > prices_trading['13d-MA-Gold']))

prices_trading['USD_Buy_Signal'] = ((prices_trading['USD_Change_3M'] < 0) & (prices_trading['USD Index Spot Rate'].diff() > 0) & (prices_trading['USD Index Spot Rate'] > prices_trading['10d-MA-Gold']))
prices_trading['USD_Sell_Signal'] = ((prices_trading['USD_Change_3M'] > 0) & (prices_trading['USD Index Spot Rate'].diff() < 0) & (prices_trading['USD Index Spot Rate'] > prices_trading['13d-MA-Gold']))

In [ ]:
# Initialize positions and portfolio variables
initial_capital = 1000000
prices_trading['Cash'] = initial_capital
prices_trading['S&P500_Position'] = 0
prices_trading['Gold_Position'] = 0
prices_trading['USD_Position'] = 0

prices_trading['S&P500_Holdings'] = 0
prices_trading['Gold_Holdings'] = 0
prices_trading['USD_Holdings'] = 0

# Allocations
allocations = {'S&P500': 0.30, 'Gold': 0.35, 'USD': 0.35}

#P&L tracker
prices_trading['S&P500_Buy_Price'] = 0
prices_trading['Gold_Buy_Price'] = 0
prices_trading['USD_Buy_Price'] = 0

prices_trading['Profit_Loss'] = 0
prices_trading['Cumulative_Profit_Loss'] = 0

In [ ]:
# Backtesting
for i in range(1, len(prices_trading)):
    if 1< prices_trading.iloc[i]['CPI YOY'] < 3 and 1 < prices_trading.iloc[i+1]['CPI YOY'] < 3 and 1 < prices_trading.iloc[i+2]['CPI YOY'] < 3:
        # Entry rules
        if prices_trading.iloc[i]['S&P500-Entry-Signal']:
            max_allocation_sp500 = allocations['S&P500'] * prices_trading.iloc[i]['Portfolio_Value']
            units_to_buy = max_allocation_sp500 // prices_trading.iloc[i]['INFLATION-ADJUSTED S&P 500']
            cost = units_to_buy * prices_trading.iloc[i]['INFLATION-ADJUSTED S&P 500']
            prices_trading.at[prices_trading.index[i], 'S&P500_Position'] = units_to_buy
            prices_trading.at[prices_trading.index[i], 'Cash'] -= cost
            prices_trading.at[prices_trading.index[i], 'S&P500_Buy_Price'] = prices_trading.iloc[i]['INFLATION-ADJUSTED S&P 500']
            
            if prices_trading.iloc[i]['Gold_Buy_Signal']:
                max_allocation_gold = allocations['Gold'] * prices_trading.iloc[i]['Portfolio_Value']
                units_to_buy = max_allocation_gold // prices_trading.iloc[i]['Gold']
                cost = units_to_buy * prices_trading.iloc[i]['Gold']
                prices_trading.at[prices_trading.index[i], 'Gold_Position'] = units_to_buy
                prices_trading.at[prices_trading.index[i], 'Cash'] -= cost
                prices_trading.at[prices_trading.index[i], 'Gold_Buy_Price'] = prices_trading.iloc[i]['Gold']
            
            if prices_trading.iloc[i]['USD_Buy_Signal']:
                max_allocation_usd = allocations['USD'] * prices_trading.iloc[i]['Portfolio_Value']
                units_to_buy = max_allocation_usd // prices_trading.iloc[i]['USD Index Spot Rate']
                cost = units_to_buy * prices_trading.iloc[i]['USD Index Spot Rate']
                prices_trading.at[prices_trading.index[i], 'USD_Position'] = units_to_buy
                prices_trading.at[prices_trading.index[i], 'Cash'] -= cost
                prices_trading.at[prices_trading.index[i], 'USD_Buy_Price'] = prices_trading.iloc[i]['USD Index Spot Rate']
                
        # Exit rules
        if prices_trading.iloc[i]['S&P500-Exit-Signal']:
            sell_value = prices_trading.iloc[i]['S&P500_Position'] * prices_trading.iloc[i]['INFLATION-ADJUSTED S&P 500']
            profit_loss = sell_value - (prices_trading.iloc[i]['S&P500_Position'] * prices_trading.iloc[i]['S&P500_Buy_Price'])
            prices_trading.at[prices_trading.index[i], 'Cash'] += sell_value
            prices_trading.at[prices_trading.index[i], 'Profit_Loss'] = profit_loss
            prices_trading.at[prices_trading.index[i], 'S&P500_Position'] = 0
            
            if prices_trading.iloc[i]['Gold_Sell_Signal']:
                sell_value = prices_trading.iloc[i]['Gold_Position'] * prices_trading.iloc[i]['Gold']
                profit_loss = sell_value - (prices_trading.iloc[i]['Gold_Position'] * prices_trading.iloc[i]['Gold_Buy_Price'])
                prices_trading.at[prices_trading.index[i], 'Cash'] += sell_value
                prices_trading.at[prices_trading.index[i], 'Profit_Loss'] += profit_loss
                prices_trading.at[prices_trading.index[i], 'Gold_Position'] = 0
                
            if prices_trading.iloc[i]['USD_Sell_Signal']:
                sell_value = prices_trading.iloc[i]['USD_Position'] * prices_trading.iloc[i]['USD Index Spot Rate']
                profit_loss = sell_value - (prices_trading.iloc[i]['USD_Position'] * prices_trading.iloc[i]['USD_Buy_Price'])
                prices_trading.at[prices_trading.index[i], 'Cash'] += sell_value
                prices_trading.at[prices_trading.index[i], 'Profit_Loss'] += profit_loss
                prices_trading.at[prices_trading.index[i], 'USD_Position'] = 0
                
    # Update holdings
    prices_trading.at[prices_trading.index[i], 'S&P500_Holdings'] = prices_trading.iloc[i]['S&P500_Position'] * prices_trading.iloc[i]['INFLATION-ADJUSTED S&P 500']
    prices_trading.at[prices_trading.index[i], 'Gold_Holdings'] = prices_trading.iloc[i]['Gold_Position'] * prices_trading.iloc[i]['Gold']
    prices_trading.at[prices_trading.index[i], 'USD_Holdings'] = prices_trading.iloc[i]['USD_Position'] * prices_trading.iloc[i]['USD Index Spot Rate']
    
    # Update portfolio value
    prices_trading.at[prices_trading.index[i], 'Portfolio_Value'] = prices_trading.iloc[i]['Cash'] + prices_trading.iloc[i]['S&P500_Holdings'] + prices_trading.iloc[i]['Gold_Holdings'] + prices_trading.iloc[i]['USD_Holdings']
    
    # Update cumulative profit/loss
    if i > 1:
        prices_trading.at[prices_trading.index[i], 'Cumulative_Profit_Loss'] = prices_trading.iloc[i-1]['Cumulative_Profit_Loss'] + prices_trading.iloc[i]['Profit_Loss']

In [ ]:
prices_trading[['Cumulative_Profit_Loss','Portfolio_Value']]